### This ended up not being used

In [ ]:
import json
from dataclasses import dataclass, field
from typing import Union

import numpy as np

In [ ]:
input_data: dict[str, dict[str, Union[dict, str]]]
with open('../../data/output/bq_no_extra_stats.json', 'r') as file:
    input_data = json.load(file)
input_data_list = input_data.values()

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
@dataclass
class DependencyCalculator:
    packages: dict[str, dict[str, Union[dict, str]]]
    dependencies: dict[str, int] = field(default_factory=dict)

    def calculate_amount_of_dependencies(self, package: dict[str, Union[dict, str]], transitive: bool):
        package_name = package['name']
        # This means we have already calculated the deps for this package
        if self.dependencies.get(package_name) is not None:
            return self.dependencies[package_name]

        self.dependencies[package_name] = 0
        for version_name, version_data in package['versions'].items():
            if len(version_data['dependencies']):
                for dependency_name in version_data['dependencies'].keys():
                    dependency = self.packages.get(dependency_name)
                    if dependency is None:
                        continue
                    if transitive:
                        self.dependencies[package_name] += self.calculate_amount_of_dependencies(dependency,
                                                                                                 transitive) + 1
                    else:
                        self.dependencies[package_name] += 1
                # if transitive:
                #     self.dependencies[package_name] /= len(version_data['dependencies'])
                break
        return self.dependencies[package_name]

    def calculate(self, transitive: bool):
        self.reset()
        for package in self.packages.values():
            if self.dependencies.get(package['name']) is None:
                self.calculate_amount_of_dependencies(package, transitive)

    def reset(self):
        self.dependencies.clear()

In [ ]:
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod() ** (1.0 / len(a))

In [ ]:
transitive_dependency_calculator = DependencyCalculator(input_data)
transitive_dependency_calculator.calculate(transitive=True)
transitive_list = list(transitive_dependency_calculator.dependencies.values())

print(np.mean(transitive_list))
print(np.median(transitive_list))
print(np.std(transitive_list))
print(np.max(transitive_list))

In [ ]:
transitive_list_no_zero = list(filter(lambda x: x > 0, transitive_list))
print(np.mean(transitive_list_no_zero))
print(np.median(transitive_list_no_zero))
print(np.std(transitive_list_no_zero))
print(np.max(transitive_list_no_zero))

In [ ]:
direct_dependency_calculator = DependencyCalculator(input_data)
direct_dependency_calculator.calculate(transitive=False)
direct_list = list(direct_dependency_calculator.dependencies.values())
print(np.mean(direct_list))
print(np.median(direct_list))
print(np.std(direct_list))
print(np.max(direct_list))